In [2]:
import os
import random
import glob
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline

In [1]:
def get_image_names(sequence, start_image, end_image):
    """
    5 char img names
    """
    image_names = []
    for image_idx in range(start_image, end_image + 1, 10):
        image_str = str(image_idx)
        while(len(image_str) < 5):
            image_str = '0' + image_str
        
        image_str += '.png'
        image_names.append(os.path.join(sequence, sequence + '_' + image_str))
    return image_names

In [3]:
def get_image_full_path(data_root, image_name, image_set, image_type):
    return os.path.join(data_root, image_set, image_type, image_name)

In [4]:
def resize_image(img_path, width, height, save_path=None):
    resized_img = Image.open(img_path)
    resized_img = resized_img.resize((width, height))
    
    if save_path is not None:
        resized_img.save(save_path)

    return resized_img

In [5]:
def category_to_id(category):
    category_dict = {
        'unlabeled' : 0,
        'ambiguous' : 1,
        'sky'       : 2,
        'road'      : 3,
        'sidewalk'  : 4,
        'railtrack' : 5,
        'terrain'   : 6,
        'tree'      : 7,
        'vegetation': 8,
        'building'  : 9,
        'infrastructure' : 10,
        'fence'          : 11,
        'billboard'      : 12,
        'trafficlight'   : 13,
        'trafficsign'    : 14,
        'mobilebarrier'  : 15,
        'firehydrant'    : 16,
        'chair'          : 17,
        'trash'          : 18,
        'trashcan'       : 19,
        'person'         : 20,
        'animal'         : 21,
        'bicycle'        : 22,
        'motorcycle'     : 23,
        'car'            : 24,
        'van'            : 25,
        'bus'            : 26,
        'truck'          : 27,
        'trailer'        : 28,
        'train'          : 29,
        'plane'          : 30,
        'boat'           : 31
    }
    return category_dict[category]

In [6]:
def create_label_image(label_image_path, categories, resize=None, save_path=None):
    
    if resize is not None:
        new_width = resize[0]
        new_height = resize[1]
        cls_image = np.array(resize_image(label_image_path, new_width, new_height, save_path=None))
    else:
        cls_image = np.array(Image.open(label_image_path))
    
    result_image = np.zeros(cls_image.shape, dtype=np.uint8)
    class_count = 1
    for category in categories:
        category_id = category_to_id(category)
        result_image += np.array(cls_image == category_id, dtype=np.uint8) * class_count

        class_count += 1
        

    out_image = Image.fromarray(result_image, 'L')
    
    if save_path is not None:
        out_image.save(save_path)
    
    return out_image
        

In [7]:
image_names  = get_image_names('075', 1170, 3200)
image_names += get_image_names('072', 10, 1590)
image_names += get_image_names('071', 10, 2660)
image_names += get_image_names('069', 10, 2410)
image_names += get_image_names('068', 1290, 4790)
image_names += get_image_names('066', 200, 2270)
image_names += get_image_names('065', 10, 560)
image_names += get_image_names('062', 1860, 2640)
image_names += get_image_names('050', 220, 2760)
image_names += get_image_names('049', 610, 3840)
image_names += get_image_names('049', 4550, 4970)
image_names += get_image_names('043', 1000, 2820)

# print(image_names)

data_root = './GTA5_dataset'
processed_data_root = os.path.join(data_root, 'sidewalk_data')
processed_data_train_rgb = os.path.join(processed_data_root, 'train', 'rgb')
processed_data_train_label = os.path.join(processed_data_root, 'train', 'label')

if not os.path.exists(processed_data_train_rgb):
    os.makedirs(processed_data_train_rgb)
if not os.path.exists(processed_data_train_label):
    os.makedirs(processed_data_train_label)

NEW_IMAGE_HEIGHT = 338
NEW_IMAGE_WIDTH  = 600
CATEGORIES = ['sidewalk', 'person', 'firehydrant', 'road']

pixel_counts = {0 : 0, 'n_image_0' : 0}
for c, category in enumerate(CATEGORIES, 1):
    cat_pix_count = {c: 0, 
                     'n_image_' + str(c) : 0}
    pixel_counts.update(cat_pix_count)

print(pixel_counts)

with open(os.path.join(processed_data_root, 'train.txt'), 'w') as file:
    for idx, image_name in enumerate(image_names):
        raw_rgb_image_path = get_image_full_path(data_root, image_name, image_set='train', image_type='img')
        raw_label_image_path = get_image_full_path(data_root, image_name, image_set='train', image_type='cls')

        output_rgb_path = os.path.join(processed_data_train_rgb, image_name)
        output_label_path = os.path.join(processed_data_train_label, image_name)

        new_image = resize_image(raw_rgb_image_path, NEW_IMAGE_WIDTH, NEW_IMAGE_HEIGHT, save_path=output_rgb_path)
        label_image = create_label_image(raw_label_image_path, CATEGORIES, resize=(NEW_IMAGE_WIDTH, NEW_IMAGE_HEIGHT), save_path=output_label_path)

        pair = '{} {}\n'.format(output_rgb_path, output_label_path)
        file.write(pair)

        values, counts = np.unique(label_image, return_counts = True)
        for value, count in zip(values, counts):
            pixel_counts[value] += count
            if (count > 0):
                pixel_counts['n_image_' + str(value)] += 1

        if idx % 100 == 0:
            print('Processing', image_name)
            print(pixel_counts)

{0: 0, 'n_image_0': 0, 2: 0, 3: 0, 'n_image_3': 0, 'n_image_1': 0, 1: 0, 'n_image_4': 0, 4: 0, 'n_image_2': 0}
Processing 075/075_01170.png
{0: 107025, 'n_image_0': 1, 2: 657, 3: 0, 'n_image_3': 0, 'n_image_1': 1, 1: 59928, 'n_image_4': 1, 4: 35190, 'n_image_2': 1}
Processing 075/075_02170.png
{0: 11896606, 'n_image_0': 101, 2: 42015, 3: 0, 'n_image_3': 0, 'n_image_1': 101, 1: 3829261, 'n_image_4': 101, 4: 4714918, 'n_image_2': 96}
Processing 075/075_03170.png
{0: 24107518, 'n_image_0': 201, 2: 156745, 3: 81, 'n_image_3': 18, 'n_image_1': 201, 1: 10108310, 'n_image_4': 201, 4: 6390146, 'n_image_2': 196}
Processing 072/072_00970.png
{0: 35490401, 'n_image_0': 301, 2: 187561, 3: 5772, 'n_image_3': 57, 'n_image_1': 298, 1: 18109165, 'n_image_4': 292, 4: 7249901, 'n_image_2': 293}
Processing 071/071_00380.png
{0: 47933397, 'n_image_0': 401, 2: 212732, 3: 14279, 'n_image_3': 147, 'n_image_1': 398, 1: 20203477, 'n_image_4': 392, 4: 12958915, 'n_image_2': 387}
Processing 071/071_01380.png
{0:

In [8]:
print(pixel_counts)
class_pixel_freqs = [pixel_counts[i] for i in range(int(len(pixel_counts) / 2))]
print('class pixel freq', class_pixel_freqs)
class_pixel_freqs = sorted(class_pixel_freqs)
print('Sorted class pixel freq', class_pixel_freqs)

freq_median = np.median(class_pixel_freqs)
print('Pixel median', freq_median)

{0: 278585757, 'n_image_0': 2369, 2: 7945393, 3: 126540, 'n_image_3': 849, 'n_image_1': 2366, 1: 120299153, 'n_image_4': 2251, 4: 73476357, 'n_image_2': 2299}
class pixel freq [278585757, 120299153, 7945393, 126540, 73476357]
Sorted class pixel freq [126540, 7945393, 73476357, 120299153, 278585757]
Pixel median 73476357.0


In [9]:
def class_freq(n_pixel, n_image_presented, image_shape):
    total_pixel = n_image_presented * image_shape[0] * image_shape[1]
    return n_pixel / total_pixel

In [10]:
classes_alpha = [class_freq(pixel_counts[i], pixel_counts['n_image_' + str(i)], (NEW_IMAGE_HEIGHT, NEW_IMAGE_WIDTH)) for i in range(int(len(pixel_counts) / 2))]
print(classes_alpha)

[0.5798636667907213, 0.2507147462990658, 0.017041525215062206, 0.0007349405147719907, 0.16095488789992088]


In [11]:
20 * 2369


47380

Creating Val Dataset

In [12]:
def get_image_names(sequence, start_image, end_image):
    image_names = []
    for image_idx in range(start_image, end_image + 1, 10):
        image_str = str(image_idx)
        while(len(image_str) < 5):
            image_str = '0' + image_str
            
        image_str += '.png'
        image_names.append(os.path.join(sequence, sequence + '_' + image_str))
    return image_names

In [13]:
image_names  = get_image_names('047', 10, 1390)
image_names += get_image_names('044', 10, 1400)
image_names += get_image_names('043', 10, 4810)
image_names += get_image_names('042', 10, 170)
image_names += get_image_names('040', 10, 1800)

# print(image_names)

data_root = './GTA5_dataset'
processed_data_root = os.path.join(data_root, 'sidewalk_data')
processed_data_val_rgb = os.path.join(processed_data_root, 'val', 'rgb')
processed_data_val_label = os.path.join(processed_data_root, 'val', 'label')

if not os.path.exists(processed_data_val_rgb):
    os.makedirs(processed_data_val_rgb)
if not os.path.exists(processed_data_val_label):
    os.makedirs(processed_data_val_label)

NEW_IMAGE_HEIGHT = 338
NEW_IMAGE_WIDTH  = 600
CATEGORIES = ['sidewalk', 'person', 'firehydrant', 'road']

pixel_counts = {0 : 0, 'n_image_0' : 0}
for c, category in enumerate(CATEGORIES, 1):
    cat_pix_count = {c: 0, 
                     'n_image_' + str(c) : 0}
    pixel_counts.update(cat_pix_count)

print(pixel_counts)

with open(os.path.join(processed_data_root, 'val.txt'), 'w') as file:
    for idx, image_name in enumerate(image_names):
        raw_rgb_image_path = get_image_full_path(data_root, image_name, image_set='val', image_type='img')
        raw_label_image_path = get_image_full_path(data_root, image_name, image_set='val', image_type='cls')

        output_rgb_path = os.path.join(processed_data_val_rgb, image_name)
        output_label_path = os.path.join(processed_data_val_label, image_name)
        
        new_image = resize_image(raw_rgb_image_path, NEW_IMAGE_WIDTH, NEW_IMAGE_HEIGHT, save_path=output_rgb_path)
        label_image = create_label_image(raw_label_image_path, CATEGORIES, resize=(NEW_IMAGE_WIDTH, NEW_IMAGE_HEIGHT), save_path=output_label_path)

        pair = '{} {}\n'.format(output_rgb_path, output_label_path)
        file.write(pair)

        values, counts = np.unique(label_image, return_counts = True)
        for value, count in zip(values, counts):
            pixel_counts[value] += count
            if (count > 0):
                pixel_counts['n_image_' + str(value)] += 1

        if idx % 100 == 0:
            print('Processing', image_name)
            print(pixel_counts)

{0: 0, 'n_image_0': 0, 2: 0, 3: 0, 'n_image_3': 0, 'n_image_1': 0, 1: 0, 'n_image_4': 0, 4: 0, 'n_image_2': 0}
Processing 047/047_00010.png
{0: 119960, 'n_image_0': 1, 2: 1022, 3: 38, 'n_image_3': 1, 'n_image_1': 1, 1: 72127, 'n_image_4': 1, 4: 9653, 'n_image_2': 1}
Processing 047/047_01010.png
{0: 12142561, 'n_image_0': 101, 2: 177561, 3: 8619, 'n_image_3': 34, 'n_image_1': 101, 1: 5069750, 'n_image_4': 98, 4: 3084309, 'n_image_2': 98}
Processing 044/044_00620.png
{0: 22706705, 'n_image_0': 201, 2: 294073, 3: 8619, 'n_image_3': 34, 'n_image_1': 201, 1: 12992722, 'n_image_4': 198, 4: 4760681, 'n_image_2': 197}
Processing 043/043_00220.png
{0: 33954019, 'n_image_0': 301, 2: 598291, 3: 10948, 'n_image_3': 56, 'n_image_1': 301, 1: 19015538, 'n_image_4': 298, 4: 7464004, 'n_image_2': 297}
Processing 043/043_01220.png
{0: 43671290, 'n_image_0': 390, 2: 785039, 3: 13788, 'n_image_3': 82, 'n_image_1': 389, 1: 23297111, 'n_image_4': 397, 4: 13555572, 'n_image_2': 385}
Processing 043/043_02220.

In [15]:
data_root = './GTA5_dataset'
processed_data_root = os.path.join(data_root, 'sidewalk_data')
processed_data_train_rgb = os.path.join(processed_data_root, 'train', 'rgb')
processed_data_train_label = os.path.join(processed_data_root, 'train', 'label')
print(processed_data_train_rgb)

./GTA5_dataset/sidewalk_data/train/rgb
